In [15]:
print("hello world")

hello world


In [ ]:

'''
this script is used to split the dataset into training and validation sets
'''

# import os, shutil
# import random

# source_dir = 'dataset'
# target_dir = 'data'
# train_ratio = 0.8  # 80% training, 20% validation

# os.makedirs(f'{target_dir}/train', exist_ok=True)
# os.makedirs(f'{target_dir}/val', exist_ok=True)

# for age in os.listdir(source_dir):
#     age_path = os.path.join(source_dir, age)
#     if not os.path.isdir(age_path):
#         continue

#     images = os.listdir(age_path)
#     random.shuffle(images)
#     split = int(train_ratio * len(images))
#     train_images = images[:split]
#     val_images = images[split:]

#     for phase, img_list in zip(['train', 'val'], [train_images, val_images]):
#         out_dir = os.path.join(target_dir, phase, age)
#         os.makedirs(out_dir, exist_ok=True)
#         for img in img_list:
#             shutil.copy(os.path.join(age_path, img), os.path.join(out_dir, img))


In [17]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [ ]:
class FaceAgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = [] 
        for age_folder in os.listdir(root_dir):
            age_path = os.path.join(root_dir, age_folder)
            if not os.path.isdir(age_path):
                continue
            try:
                age = int(age_folder)
            except ValueError:
                continue
            for fname in os.listdir(age_path):
                if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.samples.append((os.path.join(age_path, fname), age))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, age = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, age

In [ ]:
class AgeClassifierCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(AgeClassifierCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 28 * 28, 256),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
def train_model(data_dir, epochs=10, batch_size=16, lr=1e-3):

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    
    train_dataset = FaceAgeDataset(os.path.join(data_dir, 'train'), transform)
    val_dataset = FaceAgeDataset(os.path.join(data_dir, 'val'), transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AgeClassifierCNN(num_classes=100).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, ages in train_loader:
            images, ages = images.to(device), ages.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, ages)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == ages).sum().item()
            total += ages.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, ages in val_loader:
                images, ages = images.to(device), ages.to(device)
                outputs = model(images)
                loss = criterion(outputs, ages)
                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == ages).sum().item()
                val_total += ages.size(0)
        val_loss /= val_total
        val_acc = val_correct / val_total

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    
    torch.save(model.state_dict(), 'age_classifier.pth')

In [21]:
data_directory = './data'
train_model(data_directory, epochs=5, batch_size=32)

Epoch 1/5 | Train Loss: 6.2631, Train Acc: 0.0179 | Val Loss: 3.3088, Val Acc: 0.1429
Epoch 2/5 | Train Loss: 3.6542, Train Acc: 0.1250 | Val Loss: 3.5654, Val Acc: 0.1429
Epoch 3/5 | Train Loss: 3.4757, Train Acc: 0.1429 | Val Loss: 2.8414, Val Acc: 0.1429
Epoch 4/5 | Train Loss: 2.9815, Train Acc: 0.1429 | Val Loss: 2.4719, Val Acc: 0.0714
Epoch 5/5 | Train Loss: 2.5809, Train Acc: 0.1786 | Val Loss: 2.1960, Val Acc: 0.0000


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

model = AgeClassifierCNN(num_classes=100)
model.load_state_dict(torch.load('age_classifier.pth', map_location=torch.device('cpu')))
model.eval()


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


sample_image_path = 'dataset/20/image1.jpg'
image = Image.open(sample_image_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0)

with torch.no_grad():
    output = model(input_tensor)
    predicted_class = torch.argmax(output, 1).item()
    print(f'Predicted age class: {predicted_class}')

Predicted age class: 60
